# 第10章: 機械翻訳

## 90. データの準備
機械翻訳のデータセットをダウンロードせよ．訓練データ，開発データ，評価データを整形し，必要に応じてトークン化などの前処理を行うこと．ただし，この段階ではトークンの単位として形態素（日本語）および単語（英語）を採用せよ．

In [1]:
import spacy
from tqdm import tqdm

In [2]:
nlp_ja = spacy.load('ja_ginza')
nlp_en = spacy.load('en_core_web_sm')

/home/altair626/.local/lib/python3.6/site-packages/spacy/util.py:271: UserWarning: [W031] Model 'ja_ginza' (3.1.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/altair626/.local/lib/python3.6/site-packages/spacy/util.py:271: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
doc = nlp_ja('私は昨日寿司を食べました。')
for sent in doc.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_, 
              token.tag_, token.dep_, token.head.i)

0 私 私 PRON 代名詞 iobj 5
1 は は ADP 助詞-係助詞 case 0
2 昨日 昨日 ADV 名詞-普通名詞-副詞可能 advmod 5
3 寿司 寿司 NOUN 名詞-普通名詞-一般 obj 5
4 を を ADP 助詞-格助詞 case 3
5 食べ 食べる VERB 動詞-一般 ROOT 5
6 まし ます AUX 助動詞 aux 5
7 た た AUX 助動詞 aux 5
8 。 。 PUNCT 補助記号-句点 punct 5


In [ ]:
!wget -q http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz

In [ ]:
!tar xzvf kftt-data-1.0.tar.gz

In [ ]:
files = ["train","test", "dev"]

for fname in files:
    with open('./kftt-data-1.0/data/orig/kyoto-'+fname+'.ja', "r") as intxt, open("./"+fname+".ja", "w") as out:
        txts = intxt.readlines()
        for txt in tqdm(txts):
            txt = txt.strip("\n")
            doc = nlp_ja(txt)
            mrphs = [token.orth_ for sent in doc.sents for token in sent]
            print(" ".join(mrphs), file=out)

In [ ]:
files = ["train","test", "dev"]

for fname in files:
    with open('./kftt-data-1.0/data/orig/kyoto-'+fname+'.en', "r") as intxt, open("./"+fname+".en", "w") as out:
        txts = intxt.readlines()
        for txt in tqdm(txts):
            txt = txt.strip("\n")
            pt = nlp_en.make_doc(txt)
            mrphs = [doc.text for doc in pt]
            print(" ".join(mrphs), file=out)

In [4]:
!head train.ja

雪舟 （ せっ しゅう 、 1420 年 （ 応永 27 年 ） - 1506 年 （ 永正 3 年 ） ） は 号 で 、 15 世紀 後半 室町 時代 に 活躍 し た 水墨 画家 ・ 禅僧 で 、 画聖 と も 称え られる 。
日本 の 水墨画 を 一変 さ せ た 。
諱 は 「 等楊 （ とう よう ） 」 、 もしくは 「 拙 宗 （ せっ しゅう ） 」 と 号し た 。
備中 国 に 生まれ 、 京都 ・ 相国 寺 に 入っ て から 周防 国 に 移る 。
その 後 遣明 使 に 随行 し て 中国 （ 明 ） に 渡っ て 中国 の 水墨画 を 学ん だ 。
作品 は 数多く 、 中国 風 の 山水画 だけ で なく 人物画 や 花鳥画 も よく し た 。
大胆 な 構図 と 力強い 筆線 は 非常 に 個性的 な 画風 を 作り出し て いる 。
現存 する 作品 の うち 6 点 が 国宝 に 指定 さ れ て おり 、 日本 の 画家 の なか で も 別格 の 評価 を 受け て いる と いえる 。
この ため 、 花鳥 図 屏風 など に 「 伝 雪舟 筆 」 さ れる 作品 は 大変 多い 。
真筆 で ある か 専門家 の 間 で も 意見 の 分かれる もの も 多々 ある 。


In [5]:
!head train.en

Known as Sesshu ( 1420 - 1506 ) , he was an ink painter and Zen monk active in the Muromachi period in the latter half of the 15th century , and was called a master painter .
He revolutionized the Japanese ink painting .
He was given the posthumous name " Toyo " or " Sesshu ( 拙宗 ) . "
Born in Bicchu Province , he moved to Suo Province after entering SShokoku - ji Temple in Kyoto .
Later he accompanied a mission to Ming Dynasty China and learned Chinese ink painting .
His works were many , including not only Chinese - style landscape paintings , but also portraits and pictures of flowers and birds .
His bold compositions and strong brush strokes constituted an extremely distinctive style .
6 of his extant works are designated national treasures . Indeed , he is considered to be extraordinary among Japanese painters .
For this reason , there are a great many artworks that are attributed to him , such as folding screens with pictures of flowers and that birds are painted on them .
There a

## 91. 機械翻訳モデルの訓練
90で準備したデータを用いて，ニューラル機械翻訳のモデルを学習せよ（ニューラルネットワークのモデルはTransformerやLSTMなど適当に選んでよい）．

In [ ]:
!fairseq-preprocess --source-lang ja --target-lang en \
    --trainpref train \
    --validpref dev \
    --testpref test \
    --destdir ch10  \
    --workers 10

In [ ]:
! CUDA_VISIBLE_DEVICES=0,1,2,3 fairseq-train ch10 \
    --fp16 \
    --save-dir ch10-save \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --update-freq 2 \
    --dropout 0.2 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 2000 --num-workers 10 > 91.log

## 92. 機械翻訳モデルの適用
91で学習したニューラル機械翻訳モデルを用い，与えられた（任意の）日本語の文を英語に翻訳するプログラムを実装せよ．

In [ ]:
!fairseq-interactive --path ch10-save/checkpoint10.pt ch10 < test.ja | grep '^H' | cut -f3 > 92.hypothesis

## 93. BLEUスコアの計測
91で学習したニューラル機械翻訳モデルの品質を調べるため，評価データにおけるBLEUスコアを測定せよ．

In [ ]:
!fairseq-score --sys 92.hypothesis --ref test.en
#BLEU4 = 22.07, 51.7/26.9/16.3/10.5 (BP=1.000, ratio=1.087, syslen=30033, reflen=27628)

In [ ]:
!git clone https://github.com/moses-smt/mosesdecoder.git

In [ ]:
!perl ./mosesdecoder/scripts/generic/multi-bleu.perl test.en < 92.hypothesis
#Namespace(ignore_case=False, order=4, ref='test.en', sacrebleu=False, sentence_bleu=False, sys='92.hypothesis') BLEU4 = 22.07, 51.7/26.9/16.3/10.5 (BP=1.000, ratio=1.087, syslen=30033, reflen=27628)

## 94. ビーム探索
91で学習したニューラル機械翻訳モデルで翻訳文をデコードする際に，ビーム探索を導入せよ．ビーム幅を1から100くらいまで適当に変化させながら，開発セット上のBLEUスコアの変化をプロットせよ．

In [ ]:
%%bash
for N in `seq 5 15` ; do
    fairseq-interactive --path ch10-save/checkpoint10.pt --beam $N ch10 < test.ja | grep '^H' | cut -f3 > 94.out
    echo $N
    fairseq-score --sys 94.out --ref test.en >> scores94
done

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
with open("./drive/My Drive/Colab Notebooks/data/Ch10/scores94", "r") as intxt:
  scores = intxt.readlines()

scores = [i[8:13] for i in scores[1::2]]
xs = [i+1 for i in range(len(scores))]

plt.plot(xs, scores)
plt.show()
# download.png

## 95. サブワード化
トークンの単位を単語や形態素からサブワードに変更し，91-94の実験を再度実施せよ．

In [ ]:
!mkdir ch10_bpe

In [ ]:
%%bash

DIR=ch10_bpe
git clone https://github.com/rsennrich/subword-nmt.git
BPEROOT=subword-nmt/subword_nmt
BPE_TOKENS=16000
BPE_CODE=$DIR/learn_bpe.en
python $BPEROOT/learn_bpe.py -s $BPE_TOKENS < train.en > $BPE_CODE

python $BPEROOT/apply_bpe.py -c $BPE_CODE < train.en > $DIR/train.en
python $BPEROOT/apply_bpe.py -c $BPE_CODE < test.en > $DIR/test.en
python $BPEROOT/apply_bpe.py -c $BPE_CODE < dev.en > $DIR/valid.en

In [ ]:
%%bash

DIR=ch10_bpe
BPEROOT=subword-nmt/subword_nmt
BPE_TOKENS=16000
BPE_CODE=$DIR/learn_bpe.ja
python $BPEROOT/learn_bpe.py -s $BPE_TOKENS < train.ja > $BPE_CODE

python $BPEROOT/apply_bpe.py -c $BPE_CODE < train.ja > $DIR/train.ja
python $BPEROOT/apply_bpe.py -c $BPE_CODE < test.ja > $DIR/test.ja
python $BPEROOT/apply_bpe.py -c $BPE_CODE < dev.ja > $DIR/valid.ja

In [ ]:
!fairseq-preprocess --source-lang ja --target-lang en \
    --trainpref ch10_bpe/train \
    --validpref ch10_bpe/valid \
    --testpref ch10_bpe/test \
    --destdir ch10_bpe_pp  \
    --workers 5

In [ ]:
! CUDA_VISIBLE_DEVICES=0,1,2,3 fairseq-train ch10_bpe_pp \
    --fp16 \
    --save-dir ch10_bpe-save \
    --tensorboard-logdir ch10_bpe-log \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --update-freq 2 \
    --dropout 0.2 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 --num-workers 20 > 95.log

In [ ]:
!fairseq-interactive --path ch10_bpe-save/checkpoint10.pt ch10_bpe_pp < test.ja | grep '^H' | cut -f3 > 95.hypothesis

In [ ]:
!fairseq-score --sys 95.hypothesis --ref test.en
# Namespace(ignore_case=False, order=4, ref='test.en', sacrebleu=False, sentence_bleu=False, sys='95.hypothesis')
#BLEU4 = 15.56, 43.3/19.7/10.7/6.4 (BP=1.000, ratio=1.107, syslen=30596, reflen=27628)

## 96. 学習過程の可視化
Tensorboardなどのツールを用い，ニューラル機械翻訳モデルが学習されていく過程を可視化せよ．可視化する項目としては，学習データにおける損失関数の値とBLEUスコア，開発データにおける損失関数の値とBLEUスコアなどを採用せよ．

In [ ]:
!tensorboard --logdir ch10_bpe-log
# 1.PNG

## 97. ハイパー・パラメータの調整
ニューラルネットワークのモデルや，そのハイパーパラメータを変更しつつ，開発データにおけるBLEUスコアが最大となるモデルとハイパーパラメータを求めよ．

## 98. ドメイン適応
Japanese-English Subtitle Corpus (JESC)やJParaCrawlなどの翻訳データを活用し，KFTTのテストデータの性能向上を試みよ．

## 99. 翻訳サーバの構築
ユーザが翻訳したい文を入力すると，その翻訳結果がウェブブラウザ上で表示されるデモシステムを構築せよ．